## Train and deploy model on Kubeflow in Notebooks

This examples comes from a upstream fairing [example](https://github.com/kubeflow/fairing/tree/master/examples/prediction).


Please check Kaggle competiton [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)
for details about the ML problem we want to resolve.

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Amazon EKS. This notebook demonstrate how to:

* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow,
* Call the deployed endpoint for predictions.


### Install python dependencies

In [1]:
%%writefile requirements.txt
pandas
joblib
numpy
xgboost
scikit-learn>=0.21.0
seldon-core
tornado>=6.0.3
nc

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 9.5 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 103.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 58.5 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 103.8 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 80.8 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 75.3 MB/s  eta 0:00:01
     |████████████████████████████████| 77 kB 96.4 MB/s  eta 0:00:01
     |████████████████████████████████| 72 kB 33.5 MB/s  eta 0:00:01
     |████████████████████████████████| 59 kB 87.0 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 52.9 MB/s  eta 0:00:01
  Created wheel for jaeger-client: filename=jaeger_client-4.3.0-py3-none-any.whl size=65466 sha256=7a8b982643caee6d8e513a667acf633b687e320a8f9942a2b5e77d87f601dd1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1rpkkxfq/wheels/a8/37/30/d86c

  Created wheel for thrift: filename=thrift-0.13.0-cp36-cp36m-linux_x86_64.whl size=346189 sha256=c9ce7ba971871d956703cf3b0d2c22adcee2a1a5e96228e0327cdc7fb92829e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-1rpkkxfq/wheels/e0/38/fc/472fe18756b177b42096961f8bd3ff2dc5c5620ac399fce52d
Successfully built jaeger-client Flask-OpenTracing opentracing threadloop thrift
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Develop your model

In [7]:
%%writefile train_model.py
import argparse
import logging
import joblib
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

Writing train_model.py


In [8]:
%%writefile train_model.py -a
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds" %
               (model.best_score,
                model.best_iteration+1))
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    logging.info("mean_absolute_error=%.2f", mean_absolute_error(predictions, test_y))

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)
    
    
class nkTrain(object):
    
    def __init__(self):
        self.train_input = "data/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names=None):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction

Appending to train_model.py


### Train an XGBoost model in a notebook

In [6]:
model = nkTrain()
model.train()

[20:40:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:177514
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161858
[2]	validation_0-rmse:147237
[3]	validation_0-rmse:134132
[4]	validation_0-rmse:122224
[5]	validation_0-rmse:111538
[6]	validation_0-rmse:102142
[7]	validation_0-rmse:93392.3
[8]	validation_0-rmse:85824.6
[9]	validation_0-rmse:79667.6
[10]	validation_0-rmse:73463.4
[11]	validation_0-rmse:68059.4
[12]	validation_0-rmse:63350.5
[13]	validation_0-rmse:59732.1
[14]	validation_0-rmse:56260.7
[15]	validation_0-rmse:53392.6
[16]	validation_0-rmse:50770.8
[17]	validation_0-rmse:48107.8
[18]	validation_0-rmse:45923.9
[19]	validation_0-rmse:44154.2
[20]	validation_0-rmse:42488.1
[21]	validation_0-rmse:41263.3
[22]	validation_0-rmse:40212.8
[23]	validation_0-rmse:39089.1
[24]	validation_0-rmse:37691.1
[25]	validation_0-rmse:36875.2
[26]	validation_0-r

mean_absolute_error=18173.15


Best RMSE on eval: 28787.72 with 50 rounds


Model export success: trained_ames_model.dat


### Build Images and Train Remotely

> This will build image and run the training to ensure it deploys correctly
Make sure
1. you write your training files by uncommenting first line
2. Your training class or function is named nkTrain

In [10]:
!nkode create:image -d "data/train.csv"

hello auto from ./src/commands/hello.js
sh /usr/lib/node_modules/nkode/scripts/remoteTrain/remote_train.sh  auto data/train.csv tensorflow/tensorflow:1.15.0-py3
auto data/train.csv tensorflow/tensorflow:1.15.0-py3
IMDS ENDPOINT: http://169.254.169.254/
The FunctionPreProcessor is optimized for using in a notebook or IPython environment. For it to work, the python version should be same for both local python and the python in the docker. Please look at alternatives like BasePreprocessor or FullNotebookPreprocessor.
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_awhwvlma
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-sec

The job fairing-job-hg7sd launched.
Waiting for fairing-job-hg7sd-fb2m7 to start...
Waiting for fairing-job-hg7sd-fb2m7 to start...
Waiting for fairing-job-hg7sd-fb2m7 to start...
Pod started running True

  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=429305 sha256=8af63022d3bf4191cfea24edd8b2a1fecd51694ff54056294e792abb6d67de0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqqxydz5/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=45919 sha256=d948ca0787c7a6b131a764f4268a96f82cad9db0b742a07f25927dae452b0ada
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqqxydz5/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-cp36-none-any.whl size=11454 sha256=63a0e18ab535a006ba5411996e6e4bf5071d80bb28b0afec1624052d5945dca0
  Stored in directory: /tmp/pip-ephem-wheel-

Model export success: trained_ames_model.datCleaning up job fairing-job-hg7sd...

Best RMSE on eval: 29988.56 with 50 rounds


### Deploy the trained model to Kubeflow for predictions

Import the `PredictionEndpoint` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [12]:
!sh ~/nkodedemo01/nkode/remote_deploy.sh

~/nkodedemo01/notebooks/Tensor_Flow_Introduction_01/
About to deploy to end point...
The FunctionPreProcessor is optimized for using in a notebook or IPython environment. For it to work, the python version should be same for both local python and the python in the docker. Please look at alternatives like BasePreprocessor or FullNotebookPreprocessor.
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_ox5s1x1u
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-secret
Waiting for fairing-builder-nlz5b-ff22m to start...
Waiting for fairing-builder-nlz5b-ff22m to start...
Waiting for fairing-builder-nlz5b-ff22m to start...
Pod star

  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=429299 sha256=4f69e3c10cbdeadae814ef068ebfd0c2b6d5e9d66d78d04a549a2ead65e2735c
  Stored in directory: /tmp/pip-ephem-wheel-cache-58u8buqw/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-cp36-none-any.whl size=11454 sha256=8894dfc0ef1522ab62c84172affbdd06eaa86850810a8d8df87a109cd05e73bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-58u8buqw/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=45919 sha256=b985a07acdbb87fd309c0b8259aa299c9662e8fb0469fa44c3c56b6c12ccf3ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-58u8buqw/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for opentracing: filename=opentracing-2.3.0-cp36-none-any.whl size=51823 sha256=3f0de1b1298af510ef5f66912c2b